In [1]:
import numpy as np 
import pandas as pd
import os
import glob
import re
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Input, LSTM, Dense, Bidirectional
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt 
from matplotlib_venn import venn2
import seaborn as sns 


In [30]:
df = pd.read_csv("../input/darpa-timit-acousticphonetic-continuous-speech/train_data.csv")
df.head()

,index,test_or_train,dialect_region,speaker_id,filename,path_from_data_dir,path_from_data_dir_windows,is_converted_audio,is_audio,is_word_file,is_phonetic_file,is_sentence_file
0,1.0,TRAIN,DR4,MMDM0,SI681.WAV.wav,TRAIN/DR4/MMDM0/SI681.WAV.wav,TRAIN\\DR4\\MMDM0\\SI681.WAV.wav,True,True,False,False,False
1,2.0,TRAIN,DR4,MMDM0,SI1311.PHN,TRAIN/DR4/MMDM0/SI1311.PHN,TRAIN\\DR4\\MMDM0\\SI1311.PHN,False,False,False,True,False
2,3.0,TRAIN,DR4,MMDM0,SI1311.WRD,TRAIN/DR4/MMDM0/SI1311.WRD,TRAIN\\DR4\\MMDM0\\SI1311.WRD,False,False,True,False,False
3,4.0,TRAIN,DR4,MMDM0,SX321.PHN,TRAIN/DR4/MMDM0/SX321.PHN,TRAIN\\DR4\\MMDM0\\SX321.PHN,False,False,False,True,False
4,5.0,TRAIN,DR4,MMDM0,SX321.WRD,TRAIN/DR4/MMDM0/SX321.WRD,TRAIN\\DR4\\MMDM0\\SX321.WRD,False,False,True,False,False


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31678 entries, 0 to 31677
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   index                       23100 non-null  float64
 1   test_or_train               23100 non-null  object 
 2   dialect_region              23100 non-null  object 
 3   speaker_id                  23100 non-null  object 
 4   filename                    23100 non-null  object 
 5   path_from_data_dir          23100 non-null  object 
 6   path_from_data_dir_windows  23100 non-null  object 
 7   is_converted_audio          23100 non-null  object 
 8   is_audio                    23100 non-null  object 
 9   is_word_file                23100 non-null  object 
 10  is_phonetic_file            8400 non-null   object 
 11  is_sentence_file            23100 non-null  object 
dtypes: float64(1), object(11)
memory usage: 2.9+ MB


In [3]:
## Selecting only txt files
data_txt = data[(data["is_sentence_file"] == True) & (data["dialect_region"].isin(["DR1","DR2","DR3"]))]
data_txt.head()

,index,test_or_train,dialect_region,speaker_id,filename,path_from_data_dir,path_from_data_dir_windows,is_converted_audio,is_audio,is_word_file,is_phonetic_file,is_sentence_file
3403,3404.0,TRAIN,DR3,MDNS0,SX111.TXT,TRAIN/DR3/MDNS0/SX111.TXT,TRAIN\\DR3\\MDNS0\\SX111.TXT,False,False,False,False,True
3408,3409.0,TRAIN,DR3,MDNS0,SI873.TXT,TRAIN/DR3/MDNS0/SI873.TXT,TRAIN\\DR3\\MDNS0\\SI873.TXT,False,False,False,False,True
3412,3413.0,TRAIN,DR3,MDNS0,SI2271.TXT,TRAIN/DR3/MDNS0/SI2271.TXT,TRAIN\\DR3\\MDNS0\\SI2271.TXT,False,False,False,False,True
3413,3414.0,TRAIN,DR3,MDNS0,SX201.TXT,TRAIN/DR3/MDNS0/SX201.TXT,TRAIN\\DR3\\MDNS0\\SX201.TXT,False,False,False,False,True
3415,3416.0,TRAIN,DR3,MDNS0,SA1.TXT,TRAIN/DR3/MDNS0/SA1.TXT,TRAIN\\DR3\\MDNS0\\SA1.TXT,False,False,False,False,True



## 1. Use DR1 and DR2 dialect for training the model and DR3 dialect for evaluating the model.

In [32]:
df["dialect_region"].value_counts()

DR7    3850
DR3    3800
DR2    3800
DR5    3500
DR4    3400
DR1    1900
DR6    1750
DR8    1100
Name: dialect_region, dtype: int64

In [33]:
df["dialect_region"].value_counts()

DR7    3850
DR3    3800
DR2    3800
DR5    3500
DR4    3400
DR1    1900
DR6    1750
DR8    1100
Name: dialect_region, dtype: int64

In [35]:
df = df[df["dialect_region"] != 'DR4']
df = df[df["dialect_region"] != 'DR5']
df = df[df["dialect_region"] != 'DR6']
df = df[df["dialect_region"] != 'DR7']
df = df[df["dialect_region"] != 'DR8']

In [36]:
df.isnull().sum()

index                          8578
test_or_train                  8578
dialect_region                 8578
speaker_id                     8578
filename                       8578
path_from_data_dir             8578
path_from_data_dir_windows     8578
is_converted_audio             8578
is_audio                       8578
is_word_file                   8578
is_phonetic_file              13078
is_sentence_file               8578
dtype: int64

In [37]:
df = df.dropna()

In [38]:
df.isnull().sum()

index                         0
test_or_train                 0
dialect_region                0
speaker_id                    0
filename                      0
path_from_data_dir            0
path_from_data_dir_windows    0
is_converted_audio            0
is_audio                      0
is_word_file                  0
is_phonetic_file              0
is_sentence_file              0
dtype: int64

In [39]:
df["File_ext"]=df["path_from_data_dir"].str.split('.')

In [40]:
def File_extention(x):
    return x[1]

In [41]:
df.head()

,index,test_or_train,dialect_region,speaker_id,filename,path_from_data_dir,path_from_data_dir_windows,is_converted_audio,is_audio,is_word_file,is_phonetic_file,is_sentence_file,File_ext
3400,3401.0,TRAIN,DR3,MDNS0,SX291.WRD,TRAIN/DR3/MDNS0/SX291.WRD,TRAIN\\DR3\\MDNS0\\SX291.WRD,False,False,True,False,False,"[TRAIN/DR3/MDNS0/SX291, WRD]"
3401,3402.0,TRAIN,DR3,MDNS0,SX111.WAV,TRAIN/DR3/MDNS0/SX111.WAV,TRAIN\\DR3\\MDNS0\\SX111.WAV,False,True,False,False,False,"[TRAIN/DR3/MDNS0/SX111, WAV]"
3402,3403.0,TRAIN,DR3,MDNS0,SX291.PHN,TRAIN/DR3/MDNS0/SX291.PHN,TRAIN\\DR3\\MDNS0\\SX291.PHN,False,False,False,True,False,"[TRAIN/DR3/MDNS0/SX291, PHN]"
3403,3404.0,TRAIN,DR3,MDNS0,SX111.TXT,TRAIN/DR3/MDNS0/SX111.TXT,TRAIN\\DR3\\MDNS0\\SX111.TXT,False,False,False,False,True,"[TRAIN/DR3/MDNS0/SX111, TXT]"
3404,3405.0,TRAIN,DR3,MDNS0,SX21.WRD,TRAIN/DR3/MDNS0/SX21.WRD,TRAIN\\DR3\\MDNS0\\SX21.WRD,False,False,True,False,False,"[TRAIN/DR3/MDNS0/SX21, WRD]"


In [42]:
df["File_ext"]=df["File_ext"].apply(lambda x:File_extention(x) )

In [43]:
df["File_ext"]

3400    WRD
3401    WAV
3402    PHN
3403    TXT
3404    WRD
       ... 
8395    WAV
8396    WAV
8397    WAV
8398    TXT
8399    WAV
Name: File_ext, Length: 5000, dtype: object

##  2. Pick only .PHN, .WRD and .TXT files for each specified dialect distribution.

In [44]:
files=["PHN","WRD","TXT"]
df=df.loc[df['File_ext'].isin(files)]

In [45]:
df.head()

,index,test_or_train,dialect_region,speaker_id,filename,path_from_data_dir,path_from_data_dir_windows,is_converted_audio,is_audio,is_word_file,is_phonetic_file,is_sentence_file,File_ext
3400,3401.0,TRAIN,DR3,MDNS0,SX291.WRD,TRAIN/DR3/MDNS0/SX291.WRD,TRAIN\\DR3\\MDNS0\\SX291.WRD,False,False,True,False,False,WRD
3402,3403.0,TRAIN,DR3,MDNS0,SX291.PHN,TRAIN/DR3/MDNS0/SX291.PHN,TRAIN\\DR3\\MDNS0\\SX291.PHN,False,False,False,True,False,PHN
3403,3404.0,TRAIN,DR3,MDNS0,SX111.TXT,TRAIN/DR3/MDNS0/SX111.TXT,TRAIN\\DR3\\MDNS0\\SX111.TXT,False,False,False,False,True,TXT
3404,3405.0,TRAIN,DR3,MDNS0,SX21.WRD,TRAIN/DR3/MDNS0/SX21.WRD,TRAIN\\DR3\\MDNS0\\SX21.WRD,False,False,True,False,False,WRD
3406,3407.0,TRAIN,DR3,MDNS0,SX21.PHN,TRAIN/DR3/MDNS0/SX21.PHN,TRAIN\\DR3\\MDNS0\\SX21.PHN,False,False,False,True,False,PHN


In [50]:
dff["Phoneme"] = ''
dff["Phoneme"] = dff["path_from_data_dir"].apply(lambda x: add_phn(x))
dff["Word"] = ""
dff["Word"] = dff["path_from_data_dir"].apply(lambda x: add_word(x))

In [52]:
dff.columns = ["index","test_or_train","dialect_region","speaker_id",
                    "Text","Phoneme","Word"]

In [7]:
dff["Text"] = "../input/darpa-timit-acousticphonetic-continuous-speech/data/"+dff["Text"]
dff["Phoneme"] = "../input/darpa-timit-acousticphonetic-continuous-speech/data/"+dff["Phoneme"]
dff["Word"] = "../input/darpa-timit-acousticphonetic-continuous-speech/data/"+dff["Word"]
dff.head()

,index,test_or_train,dialect_region,speaker_id,Text,Phoneme,Word
3403,3404.0,TRAIN,DR3,MDNS0,../input/darpa-timit-acousticphonetic-continuo...,../input/darpa-timit-acousticphonetic-continuo...,../input/darpa-timit-acousticphonetic-continuo...
3408,3409.0,TRAIN,DR3,MDNS0,../input/darpa-timit-acousticphonetic-continuo...,../input/darpa-timit-acousticphonetic-continuo...,../input/darpa-timit-acousticphonetic-continuo...
3412,3413.0,TRAIN,DR3,MDNS0,../input/darpa-timit-acousticphonetic-continuo...,../input/darpa-timit-acousticphonetic-continuo...,../input/darpa-timit-acousticphonetic-continuo...
3413,3414.0,TRAIN,DR3,MDNS0,../input/darpa-timit-acousticphonetic-continuo...,../input/darpa-timit-acousticphonetic-continuo...,../input/darpa-timit-acousticphonetic-continuo...
3415,3416.0,TRAIN,DR3,MDNS0,../input/darpa-timit-acousticphonetic-continuo...,../input/darpa-timit-acousticphonetic-continuo...,../input/darpa-timit-acousticphonetic-continuo...


In [54]:
df_train= dff[dff["dialect_region"] != 'DR3']

In [55]:
df_test= dff[dff["dialect_region"] != 'DR1']
df_test= df_test[df_test["dialect_region"] != 'DR2']

In [56]:
df_train.head()

,index,test_or_train,dialect_region,speaker_id,Text,Phoneme,Word
7200,7201.0,TRAIN,DR2,FJKL0,TRAIN/DR2/FJKL0/SX32.PHN,TRAIN/DR2/FJKL0/SX32.PHN,TRAIN/DR2/FJKL0/SX32.WRD
7202,7203.0,TRAIN,DR2,FJKL0,TRAIN/DR2/FJKL0/SX32.WRD,TRAIN/DR2/FJKL0/SX32.PHN,TRAIN/DR2/FJKL0/SX32.WRD
7203,7204.0,TRAIN,DR2,FJKL0,TRAIN/DR2/FJKL0/SI932.WRD,TRAIN/DR2/FJKL0/SI932.PHN,TRAIN/DR2/FJKL0/SI932.WRD
7204,7205.0,TRAIN,DR2,FJKL0,TRAIN/DR2/FJKL0/SX302.TXT,TRAIN/DR2/FJKL0/SX302.PHN,TRAIN/DR2/FJKL0/SX302.WRD
7205,7206.0,TRAIN,DR2,FJKL0,TRAIN/DR2/FJKL0/SX122.WRD,TRAIN/DR2/FJKL0/SX122.PHN,TRAIN/DR2/FJKL0/SX122.WRD


In [8]:
Train = data_txt[(data_txt["dialect_region"] == "DR1")|
            (data_txt["dialect_region"] == "DR2")]
Test = data_txt[data_txt["dialect_region"] == "DR3"]

In [57]:
def words(dir):
  w=[]
  phn = []
  dir=dir.split('.')[0]
  dict_file_wrd = open(str(dir+".WRD"), 'r')
  dict_file_phn = open(str(dir+".PHN"), 'r')
  with dict_file_wrd as f:
    phonics = [line.rstrip('\n') for line in f]
    for p in phonics:
      p=p.split(' ')
      s="".join(p[2:])
      w.append(s)
  with dict_file_phn as f:
    phonics = [line.rstrip('\n') for line in f]
    for p in phonics:
      p=p.split(' ')
      s="".join(p[2:])
      phn.append(s)
  return w,phn

In [12]:
train_vocab = {}
test_vocab = {}
for word,phoneme in zip(train_w,train_p):
    train_vocab[word] = phoneme
for word,phoneme in zip(test_w,test_p):
    test_vocab[word] = phoneme

## 3. Create a vocabulary for each word with its associated phonemes.



In [13]:
vocab = {**train_vocab,**test_vocab}

In [15]:
Train_vocab = pd.DataFrame({"Word":train_vocab.keys(),"Phoneme":train_vocab.values()})
Test_vocab = pd.DataFrame({"Word":test_vocab.keys(),"Phoneme":test_vocab.values()})
Vocab = pd.DataFrame({"Word":vocab.keys(),"Phoneme":vocab.values()})

In [16]:
Vocab

,Word,Phoneme
0,tofu,t ow f uw
1,is,ix z
2,made,m ey dx
3,from,f axr m
4,processed,pcl p r aa s eh s
...,...,...
3089,pastime,p ae s tcl t ay m
3090,hooking,hh uh kcl k ix ng
3091,braiding,bcl b r ey dx iy ng
3092,rugs,epi r ah gcl g z


In [19]:
Train_file = pd.DataFrame()
Train_file["Sentence"] = train_sentence
Train_file["Phoneme"] = train_phoneme

Test_file = pd.DataFrame()
Test_file["Sentence"] = test_sentence
Test_file["Phoneme"] = test_phoneme


In [22]:
data = Train_file.append(Test_file)
data.reset_index(inplace=True)

## 4. Train the model with available vocabulary for phoneme to word prediction. (Use sequence modeling).



In [23]:
phoneme_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<OOV>')
phoneme_tokenizer.fit_on_texts(Vocab['Phoneme'].values)
phoneme_index = phoneme_tokenizer.word_index
print(phoneme_index)

{'<OOV>': 1, 's': 2, 'ix': 3, 'n': 4, 'tcl': 5, 'l': 6, 'kcl': 7, 'r': 8, 'k': 9, 'iy': 10, 'dcl': 11, 't': 12, 'ih': 13, 'm': 14, 'eh': 15, 'z': 16, 'pcl': 17, 'p': 18, 'd': 19, 'ae': 20, 'ax': 21, 'axr': 22, 'ey': 23, 'aa': 24, 'dx': 25, 'f': 26, 'b': 27, 'er': 28, 'bcl': 29, 'ay': 30, 'ah': 31, 'v': 32, 'ow': 33, 'q': 34, 'ng': 35, 'gcl': 36, 'ao': 37, 'w': 38, 'sh': 39, 'g': 40, 'el': 41, 'jh': 42, 'ux': 43, 'ch': 44, 'y': 45, 'epi': 46, 'nx': 47, 'en': 48, 'hv': 49, 'aw': 50, 'pau': 51, 'th': 52, 'uw': 53, 'hh': 54, 'dh': 55, 'oy': 56, 'uh': 57, 'h': 58, 'zh': 59, 'em': 60, 'eng': 61}


In [24]:
phoneme_length = len(phoneme_tokenizer.word_index) + 1
phoneme_length

62

In [25]:
train_sequences = phoneme_tokenizer.texts_to_sequences(Train_vocab['Phoneme'].values)
test_sequences = phoneme_tokenizer.texts_to_sequences(Test_vocab['Phoneme'].values)

In [26]:
longest_phoneme = len(max(train_sequences, key=len))
longest_phoneme

17

In [27]:
train_padded = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=longest_phoneme, padding='post')
test_padded = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=longest_phoneme, padding='post')

print("Word: {}\n Padded:{}\n phoneme:{}".format(Train_vocab["Word"][0],train_padded[0],Train_vocab["Phoneme"][0]))

Word: tofu
 Padded:[12 33 26 43  0  0  0  0  0  0  0  0  0  0  0  0  0]
 phoneme:t ow f ux


In [28]:
y = Vocab['Word'].values
y_train = Train_vocab['Word'].values
y_test = Test_vocab["Word"].values
                      
encoder = LabelEncoder()
encoder.fit(y)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)
                      

In [29]:
embedding_dim = 10
tf.keras.backend.clear_session()
model = Sequential([
    Embedding(phoneme_length, embedding_dim, 
                              input_length=longest_phoneme),
    Bidirectional(LSTM(50)),
    Dense(128, activation='relu'),
    Dense(len(Vocab['Word'].unique()), activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.summary()

2022-09-23 01:56:50.689092: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-23 01:56:50.791524: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-23 01:56:50.792742: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-23 01:56:50.794990: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 17, 10)            620       
_________________________________________________________________
bidirectional (Bidirectional (None, 100)               24400     
_________________________________________________________________
dense (Dense)                (None, 128)               12928     
_________________________________________________________________
dense_1 (Dense)              (None, 3094)              399126    
Total params: 437,074
Trainable params: 437,074
Non-trainable params: 0
_________________________________________________________________


In [40]:
history = model.fit(train_padded,y_train,
                    batch_size=32,
                    epochs=50)

Epoch 1/50

75/75 [==============================] - 7s 28ms/step - loss: 8.0484

Epoch 2/50

75/75 [==============================] - 2s 28ms/step - loss: 8.0239

Epoch 3/50

75/75 [==============================] - 2s 27ms/step - loss: 8.0082

Epoch 4/50

75/75 [==============================] - 2s 27ms/step - loss: 7.9765

Epoch 5/50

75/75 [==============================] - 2s 27ms/step - loss: 7.8738

Epoch 6/50

75/75 [==============================] - 2s 28ms/step - loss: 7.5782

Epoch 7/50

75/75 [==============================] - 2s 27ms/step - loss: 7.0607

Epoch 8/50

75/75 [==============================] - 2s 27ms/step - loss: 6.3292

Epoch 9/50

75/75 [==============================] - 2s 27ms/step - loss: 5.6117

Epoch 10/50

75/75 [==============================] - 2s 27ms/step - loss: 4.9979

Epoch 11/50

75/75 [==============================] - 3s 36ms/step - loss: 4.4618

Epoch 12/50

75/75 [==============================] - 2s 28ms/step - loss: 3.9993

Epoch 13/50



## 5. Create a pandas dataframe containing the sentence phonemes for each file.
## 6. Predict the words of all phonemes present in the sentence using the trained model.

In [41]:
def predict_words(sentence):
    phonemes = sentence.split("  ")
    #print(phonemes)
    word_sentence = ""
    for x in phonemes:
        text_seq = phoneme_tokenizer.texts_to_sequences([x])
        text_padded = pad_sequences(text_seq, maxlen=longest_phoneme, padding='post')
        pred = model.predict(text_padded)
        pred = np.argmax(pred,axis=1)
        word = encoder.classes_[pred[0]]
        #print(word)
        word_sentence = word_sentence + word + " "
    word_sentence = word_sentence.strip()
    return word_sentence

In [43]:
data["predicted_sentence"] = data["Phoneme"].apply(predict_words)
data.head()

,index,Sentence,Phoneme,split,predicted_sentence
0,0,tofu is made from processed soybeans,t ow f uw ih z m ey dcl f r em pcl p r aa ...,Train,author's how had fresh process soybeans
1,1,i gave them several choices and let the...,ay gcl g ey v dh ah m s eh v r el tcl ch o...,Train,are gave warm speak atmosphere annoying let th...
2,2,she had your dark suit in greasy wash ...,sh iy hv ae dcl d y axr dcl d aa r kcl k s...,Train,she hood y'all dessert suit eat greasing wash ...
3,3,don't ask me to carry an oily rag like...,d ow n ae s kcl m iy dx ih kcl k eh r iy ...,Train,age waste who it carry an oily rag work gait
4,4,a young mouse scampered across the field...,q ax y ah ng m aw s kcl k ae m pcl p axr dx...,Train,al young mask scampered decking over reason an...


## 7. Calculate the distance between predicted and actual words for each sentence.
## 8. Apply the same process in the testing set, provide your inferences.


In [60]:

def sent_distance(sent1,sent2):
    distance_word = []
    for w1,w2 in zip(sent1.split(),sent2.split()):
        dist = word_distances(w1,w2)
        distance_word.append(str(dist))
    a = ", ".join(distance_word)
    return a

In [46]:
def w_distances(w1,w2):
    x1 = w2v.wv.get_vector(w1)
    x2 = w2v.wv.get_vector(w2)
    distance = euclidean(x1,x2)
    return round(distance,2)

In [59]:
def t_distance(sent1,sent2):
    total_distance = 0
    for w1,w2 in zip(sent1.split(),sent2.split()):
        dist = word_distances(w1,w2)
        total_distance = total_distance + dist
    return total_distance

In [47]:
t_distance("tofu  is  made  from  processed  soybeans","roles inch voice late process soybeans")

0.7400000000000001

In [48]:
data["total_distance"] = data.apply(lambda x: t_distance(x['Sentence'],x["predicted_sentence"]),axis = 1)
data["word_distance"] = data.apply(lambda x: w_distance(x['Sentence'],x["predicted_sentence"]),axis = 1)
data.head()

,index,Sentence,Phoneme,split,predicted_sentence,total_distance,word_distance
0,0,tofu is made from processed soybeans,t ow f uw ih z m ey dcl f r em pcl p r aa ...,Train,author's how had fresh process soybeans,0.72,"0.13, 0.16, 0.15, 0.12, 0.16, 0.0"
1,1,i gave them several choices and let the...,ay gcl g ey v dh ah m s eh v r el tcl ch o...,Train,are gave warm speak atmosphere annoying let th...,1.13,"0.13, 0.0, 0.17, 0.15, 0.13, 0.15, 0.0, 0.0, 0..."
2,2,she had your dark suit in greasy wash ...,sh iy hv ae dcl d y axr dcl d aa r kcl k s...,Train,she hood y'all dessert suit eat greasing wash ...,1.09,"0.0, 0.14, 0.16, 0.14, 0.0, 0.11, 0.12, 0.0, 0..."
3,3,don't ask me to carry an oily rag like...,d ow n ae s kcl m iy dx ih kcl k eh r iy ...,Train,age waste who it carry an oily rag work gait,0.79,"0.14, 0.13, 0.15, 0.12, 0.0, 0.0, 0.0, 0.0, 0...."
4,4,a young mouse scampered across the field...,q ax y ah ng m aw s kcl k ae m pcl p axr dx...,Train,al young mask scampered decking over reason an...,0.97,"0.15, 0.0, 0.16, 0.0, 0.14, 0.12, 0.14, 0.12, ..."


In [51]:
for i in range(10,15):
    print("Actual Sentence:",data["Sentence"][i])
    print("Predicted Sentence: ",data["predicted_sentence"][i])
    print("Word distance: ",data["word_distance"][i])
    print("Total distance: ",data["total_distance"][i])
    print("\n")

Actual Sentence: solid  concrete  blocks  relatively  heavy  and  dense  are  used  for  this  shelter

Predicted Sentence:  solid concrete blocks relatively heavy age dense 'em without full they shelter

Word distance:  0.0, 0.0, 0.0, 0.0, 0.0, 0.13, 0.0, 0.1, 0.14, 0.14, 0.1, 0.0

Total distance:  0.61





Actual Sentence: brush  fires  are  common  in  the  dry  underbrush  of  nevada

Predicted Sentence:  brush fires pie common inch over dry fatigues bob numbness

Word distance:  0.0, 0.0, 0.16, 0.0, 0.18, 0.12, 0.0, 0.18, 0.13, 0.13

Total distance:  0.8999999999999999





Actual Sentence: norwegian  sweaters  are  made  of  lamb's  wool

Predicted Sentence:  informative surprise art here of lamb's wool

Word distance:  0.15, 0.16, 0.17, 0.12, 0.0, 0.0, 0.0

Total distance:  0.6





Actual Sentence: she  had  your  dark  suit  in  greasy  wash  water  all  year

Predicted Sentence:  she tim y'all dessert suit kind greasing wash tiny all after

Word distance:  0.0, 0.13, 0.16, 0